In [2]:
import os
from bs4 import BeautifulSoup
import random
import nltk
from nltk.corpus import stopwords
import string
!pip install contractions
import contractions
import pickle
nltk.download('stopwords')
import numpy as np
import math
from tqdm import tqdm

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Moksh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Data Preprocessing

In [3]:
path = "../data/CSE508_Winter2023_Dataset"
dir_ = os.listdir(path)
doc_id_to_name = {}  # Mapping doc_id numbers to the documents names
file_data = {}  # Document data according to document name

for i in tqdm(range(len(dir_))):
    doc_id_to_name[i] = dir_[i]

for i in tqdm(range(len(dir_))):
    with open(f'{path}/{dir_[i]}') as f:
        contents = f.read()
        file_data[dir_[i]] = contents

100%|██████████████████████████████████████████████████████████████████████████████| 1400/1400 [00:23<00:00, 59.60it/s]


In [4]:
doc_id_to_name[34]

'cranfield0035'

# Extracting Relevant Text

In [5]:
req_text = {}  # Required data between these mentioned tags

for key in file_data.keys():
    text = file_data[key]
    
    soup = BeautifulSoup(text, 'html.parser')  # Creating soup object and passing it the text file
    title_tag = soup.find('title')  # Searching for TITLE tag in the document
    text_tag = soup.find('text')  # Searching for TEXT tag in the document
    
    title_data = ""
    text_data = ""
    
    if title_tag:
        title_data = title_tag.text  # If TITLE tag is found updating the title data otherwise it is empty
    
    if text_tag:
        text_data = text_tag.text  # If TEXT tag is found updating the text data otherwise it is empty
        
    new_text = f'{title_data} {text_data}'  # Concatenating the data with a blank space
    
    req_text[key] = new_text

**Printing for first 5 files**

In [6]:
# Printing the content for first 5 files before and after relevant extraction
for i in range(5):
    print("Before !-------------------!")
    print()
    print(file_data[doc_id_to_name[i]])
    print()
    print("After  !-------------------!")
    print()
    print(req_text[doc_id_to_name[i]])
    print()

Before !-------------------!

<DOC>
<DOCNO>
1
</DOCNO>
<TITLE>
experimental investigation of the aerodynamics of a
wing in a slipstream .
</TITLE>
<AUTHOR>
brenckman,m.
</AUTHOR>
<BIBLIO>
j. ae. scs. 25, 1958, 324.
</BIBLIO>
<TEXT>
  an experimental study of a wing in a propeller slipstream was
made in order to determine the spanwise distribution of the lift
increase due to slipstream at different angles of attack of the wing
and at different free stream to slipstream velocity ratios .  the
results were intended in part as an evaluation basis for different
theoretical treatments of this problem .
  the comparative span loading curves, together with supporting
evidence, showed that a substantial part of the lift increment
produced by the slipstream was due to a /destalling/ or boundary-layer-control
effect .  the integrated remaining lift increment,
after subtracting this destalling lift, was found to agree
well with a potential flow theory .
  an empirical evaluation of the destalling 

# Preprocessing

In [7]:
proc_text = {}  # Storing the processed text

# Converting the passed text to lower case
def to_lower(text):
    new_text = text.lower()
    new_text = contractions.fix(new_text)  # contractions library is used to remove the contracted form of words. Eg - "I'll" will be converted to "I will" etc.
    return new_text

# Tokenizing the passed text
def tokenize(text):
    tokens = nltk.tokenize.word_tokenize(text)
    return tokens

# Removing stop words from the tokens passed. Eg - a, an, the, and etc.
def remove_stopwords(tokens):
    stop_words = set(stopwords.words("english"))
    res = []
    for word in tokens:
        if word not in stop_words:
            res.append(word)
    return res

# Removing the punctuations from the tokens passed by iterating on a character level. Eg - . , ! - etc.
def remove_punc(tokens):
    punctuations = string.punctuation
    res = []
    for word in tokens:
        new_word = ""
        for character in word:
            if character not in punctuations:
                new_word += character
        res.append(new_word)
    
    return res

# Removing blank space characters ("") if any from the passed tokens 
def remove_blank(tokens):
    res = []
    for word in tokens:
        if word != "":
            res.append(word)
            
    return res

def print_text(text, i, operation):
    if i < 5:
        print(operation)
        print("-----------------")
        print(text)
        print()

i = 0
# Performing the above defined preprocessing steps for all the file of the above dataset
for key in req_text.keys():
    text = req_text[key]
    print_text(text, i, "Original Text")
    text = to_lower(text)  # Lower casing the text
    print_text(text, i, "To lowercase")
    text = tokenize(text)  # Tokenizing
    print_text(text, i, "Tokenized Text")
    text = remove_stopwords(text)  # Removing stop words
    print_text(text, i, "Removed Stopwords")
    text = remove_punc(text)  # Removing punctuation marks
    print_text(text, i, "Removed Punctuations")
    text = remove_blank(text)  # Removing blank spaces
    print_text(text, i, "Removed Blankspaces")
    
    i += 1
    proc_text[key] = text

Original Text
-----------------

experimental investigation of the aerodynamics of a
wing in a slipstream .
 
  an experimental study of a wing in a propeller slipstream was
made in order to determine the spanwise distribution of the lift
increase due to slipstream at different angles of attack of the wing
and at different free stream to slipstream velocity ratios .  the
results were intended in part as an evaluation basis for different
theoretical treatments of this problem .
  the comparative span loading curves, together with supporting
evidence, showed that a substantial part of the lift increment
produced by the slipstream was due to a /destalling/ or boundary-layer-control
effect .  the integrated remaining lift increment,
after subtracting this destalling lift, was found to agree
well with a potential flow theory .
  an empirical evaluation of the destalling effects was made for
the specific configuration of the experiment .


To lowercase
-----------------

experimental investi



To lowercase
-----------------

approximate solutions of the incompressible laminar
boundary layer equations for a plate in shear flow .
 
  the two-dimensional steady boundary-layer
problem for a flat plate in a
shear flow of incompressible fluid is considered .
solutions for the boundarylayer
thickness, skin friction, and the velocity
distribution in the boundary
layer are obtained by the karman-pohlhausen
technique .  comparison with
the boundary layer of a uniform flow has also
been made to show the effect of
vorticity .


Tokenized Text
-----------------
['approximate', 'solutions', 'of', 'the', 'incompressible', 'laminar', 'boundary', 'layer', 'equations', 'for', 'a', 'plate', 'in', 'shear', 'flow', '.', 'the', 'two-dimensional', 'steady', 'boundary-layer', 'problem', 'for', 'a', 'flat', 'plate', 'in', 'a', 'shear', 'flow', 'of', 'incompressible', 'fluid', 'is', 'considered', '.', 'solutions', 'for', 'the', 'boundarylayer', 'thickness', ',', 'skin', 'friction', ',', 'and', 'the

In [9]:
# Storing the total unique words throught the complete document dataset and assigning them unique term ids
vocabulary = []
id_to_term = {}

for doc in proc_text.keys():
    for word in proc_text[doc]:
        vocabulary.append(word)

print(len(vocabulary))
vocabulary = list(set(vocabulary))
print(len(vocabulary))

for i, term in enumerate(vocabulary):
    id_to_term[i] = term
    

k = 10
print(id_to_term[k])
print(vocabulary[k])

127208
8990
threshold
threshold


# TF-IDF Matrix

In [15]:
weight_schemes = ['binary', 'raw_count', 'term_frequency', 'log_normal', 'double_normal'] # All different weighting schemes for tf weights

# Computing the frequencies of all unique terms in the dataset
def raw_term_frequency():
    rtf = {}
    for i in range(len(proc_text)):
        rtf[i] = {}
        unique_words, freq = np.unique(proc_text[doc_id_to_name[i]], return_counts = True)
        for j in range(len(unique_words)):
            rtf[i][unique_words[j]] = freq[j]
    
    return rtf

# Posting list for all the terms Eg - ('experiment' = [1, 2, 16, ...], ...)
def postings_list():
    post_list = {}
    for i in range(len(proc_text)):
        words = np.unique(proc_text[doc_id_to_name[i]])
        for word in words:
            if word not in post_list.keys():
                post_list[word] = [i]
            else:
                if i not in post_list[word]:
                    post_list[word].append(i)
                
    return post_list

# Computing the idf score for each term after getting the postings list
# idf score of term = log10(total number of documents / document frequency(term) + 1)
def compute_idf(post_list):
    idf = {}
    for term in post_list.keys():
        idf[term] = math.log10(len(proc_text) / (len(post_list[term]) + 1))
        
    return idf

# Computing the term frequency weights of all terms for all the documents
# Weighting schemes = ['Binary', 'Raw count', 'Term frequency', 'Log normalization', 'Double normalization']
def compute_tf_weight(scheme, term, doc_tf):
    # Return 1 if term found in document else 0
    if scheme == "binary":
        if term in doc_tf.keys():
            return 1
        else:
            return 0
    # Return raw count of term in document  
    if scheme == "raw_count":
        if term in doc_tf.keys():
            return doc_tf[term]
        else:
            return 0
    # Return frequency of term in document / total length of documents tokens
    if scheme == "term_frequency":
        if term in doc_tf.keys():
            return doc_tf[term] / sum(doc_tf.values())
        else:
            return 0
    # Return log10(1 + frequency of term in document)
    if scheme == "log_normal":
        if term in doc_tf.keys():
            return math.log10(1 + doc_tf[term])
        else:
            return 0
    # Return 0.5(1 + frequency of term in document / max frequency of vocabulary in document dataset)
    if scheme == "double_normal":
        if term in doc_tf.keys():
            return 0.5 + 0.5 * doc_tf[term] / max(doc_tf.values())
        else:
            return 0.5
        
# Generating the tf_idf weights matrix 
def compute_tf_idf_matrix():
    # Initialising the tf-idf matrix for all weighting schemes 
    tf_idf = {}
    rtf = raw_term_frequency() # Fetching the raw frequencies for all the terms
    post_list = postings_list() # Computing postings list for all the terms
    term_idf = compute_idf(post_list) # Computing the idf scores for all the terms
    for scheme in weight_schemes:
        print(f'Computing weights for {scheme} scheme -> -> ->')
        tf_idf[scheme] = {}
        tf_idf[scheme] = np.zeros((len(proc_text), len(vocabulary))) # Declaring the numpy zeros array of dimensions (number of documents, number of terms in vocabulary)
        for doc in tqdm(range(len(proc_text))):
            for j in range(len(vocabulary)):
                tf_weight = compute_tf_weight(scheme, vocabulary[j], rtf[doc]) # tf weight of term j in document i for the given scheme
                idf = term_idf[vocabulary[j]] # idf score for term j
                tf_idf[scheme][doc][j] = tf_weight * idf # Tf-idf score for term j in document i for given scheme
    
    return tf_idf

# Preprocessing the query passed
def query_preprocessing(text):
    text = to_lower(text)  # Lower casing the text
    text = tokenize(text)  # Tokenizing
    text = remove_stopwords(text)  # Removing stop words
    text = remove_punc(text)  # Removing punctuation marks
    text = remove_blank(text)  # Removing blank spaces
    return text

# Computing the query vector for given query tokens
def compute_query_vector(query_tokens):
    query_vector = {}
    query_tf = {} # Term frequencies for query tokens in document dataset
    term_idf = compute_idf(postings_list()) # idf scores for terms
    for term in query_tokens:
        if term not in query_tf.keys():
            query_tf[term] = 1
        else:
            query_tf[term] += 1
    
    print(query_tf)
    for scheme in weight_schemes:
        # Declaring query vector as numpy zeros of dimensions (length of vocabulary, 1)
        query_vector[scheme] = np.zeros((len(vocabulary), 1)) 
        for i in range(len(vocabulary)):
            tf_weight_ = compute_tf_weight(scheme, vocabulary[i], query_tf) # Assiging the required tf weight for ith vocabulary term
            idf_ = term_idf[vocabulary[i]] if term in term_idf.keys() else 0
            query_vector[scheme][i] = tf_weight_ * idf_ # Tf-idf score for qery token
            
    return query_vector

# Generate results for the query passed for all weighting schemes and return top 5 documents for each
def solve_tf_idf_query(query, tf_idf):
    query_tokens = query_preprocessing(query) # Generating query tokens 
    query_vector = compute_query_vector(query_tokens) # Compute query vector for the processed query
    print(query) 
    print(query_tokens)
    print()
    
    for scheme in weight_schemes:
        print(f"Results for scheme : {scheme}")
        tf_mat = tf_idf[scheme] # Tf-idf matrix for the given scheme
        query_vec = query_vector[scheme] # Getting the query vector for the given scheme
        mult = np.dot(tf_mat, query_vec).reshape(len(proc_text)) # Getting the matrix multiplication for tf-idf matrix and query vector
        ans = (-mult).argsort()[:5] # Getting the documents with top 5 scores
        for ind in range(len(ans)):
            print(doc_id_to_name[ans[ind]], end = " ") # Priting the document names and scores for the best 5 documents
        print()
        vals = []
        for idx in ans:
            vals.append(mult[idx])
        print(vals)
        print()
        print("----------------------")
        print()

In [83]:
# tf_idf = compute_tf_idf_matrix()

  0%|▏                                                                                | 3/1400 [00:00<01:07, 20.70it/s]

Computing weights for binary scheme -> -> ->


  0%|▏                                                                                | 3/1400 [00:00<00:48, 29.00it/s]

Computing weights for raw_count scheme -> -> ->


  0%|▏                                                                                | 3/1400 [00:00<00:50, 27.76it/s]

...

  0%|                                                                                 | 2/1400 [00:00<01:10, 19.81it/s]

Computing weights for log_normal scheme -> -> ->


  0%|▏                                                                                | 3/1400 [00:00<00:50, 27.73it/s]

Computing weights for double_normal scheme -> -> ->


100%|██████████████████████████████████████████████████████████████████████████████| 1400/1400 [00:53<00:00, 26.33it/s]


In [112]:
# for scheme in weight_schemes:
#     pickle.dump(tf_idf[scheme], open(f"../Q1/{scheme}_tf_idf.pkl", 'wb'))

In [11]:
tf_idf = {}
for scheme in weight_schemes:
    tf_idf[scheme] = pickle.load(open(f"../Q1/{scheme}_tf_idf.pkl", 'rb'))

In [12]:
print(type(tf_idf['binary']))

<class 'numpy.ndarray'>


In [17]:
solve_tf_idf_query("Experimental design of aerodynamic flows", tf_idf)

{'experimental': 1, 'design': 1, 'aerodynamic': 1, 'flows': 1}
Experimental design of aerodynamic flows
['experimental', 'design', 'aerodynamic', 'flows']

Results for scheme : binary
cranfield1285 cranfield0811 cranfield0518 cranfield0219 cranfield1088 
[2.6956477391845755, 2.5680012843948976, 2.5680012843948976, 2.5680012843948976, 1.8352839030468309]

----------------------

Results for scheme : raw_count
cranfield0094 cranfield0811 cranfield1386 cranfield1226 cranfield0721 
[6.785037002184044, 5.136002568789795, 3.392518501092022, 3.392518501092022, 3.392518501092022]

----------------------

Results for scheme : term_frequency
cranfield1285 cranfield0470 cranfield0389 cranfield0939 cranfield0460 
[0.019254626708461252, 0.015145171879875099, 0.012472494489308903, 0.007853052085861161, 0.007439733555026364]

----------------------

Results for scheme : log_normal
cranfield0811 cranfield0094 cranfield1285 cranfield1386 cranfield1226 
[0.36883639860174133, 0.35691149893759483, 0.24427

# Jacardian Coefficient

In [18]:
# Intersection for query tokens and document tokens
def intersection(query_tokens, doc_tokens):
    return len(list(set(query_tokens) & set(doc_tokens)))

# Union for query tokens and document tokens
def union(query_tokens, doc_tokens):
    return len(list(set(query_tokens) | set(doc_tokens)))

# Gererating jacardian coefficient results for the given query
def solve_jacardian_query(query):
    query_tokens = query_preprocessing(query) # Getting query tokens
    print(query_tokens)
    print()
    coeff = [0] * len(proc_text) # Jacardian Coefficient array for all the documents
    for i in range(len(coeff)):
        intersection_ = intersection(query_tokens, proc_text[doc_id_to_name[i]]) # Number of intersections for query tokens and document tokens
        union_ = union(query_tokens, proc_text[doc_id_to_name[i]]) # Number of unions for query tokens and document tokens
        coeff[i] =  intersection_ / union_ # Jacardian coefficient for given query and document dataset
    coeff = np.array(coeff)
    ans = (-coeff).argsort()[:5] # Getting the top 5 documents with highest scores
    for ind in range(len(ans)): # Printing details of these 5 best documents
        print(doc_id_to_name[ans[ind]], end = " ")
    print()
    vals = []
    for idx in ans:
        vals.append(coeff[idx])
    print(vals)

In [19]:
solve_jacardian_query("Experimental design of experimental flows")

['experimental', 'design', 'experimental', 'flows']

cranfield1045 cranfield0418 cranfield0935 cranfield1069 cranfield0203 
[0.06666666666666667, 0.0625, 0.0625, 0.06060606060606061, 0.05714285714285714]


In [21]:
# Main function for running question 1
def main():
    print("Enter 0 -> TF-IDF Query")
    print("Enter 1 -> Jacardian Query")
    n = int(input())
    print("Enter Query")
    query = str(input())
    if query == "":
        print("Empty query")
        return
    if n == 0:
        solve_tf_idf_query(query, tf_idf)
    else:
        solve_jacardian_query(query)

main()
        

Enter 0 -> TF-IDF Query
Enter 1 -> Jacardian Query
0
Enter Query
experimental design for experimental flows
{'experimental': 2, 'design': 1, 'flows': 1}
experimental design for experimental flows
['experimental', 'design', 'experimental', 'flows']

Results for scheme : binary
cranfield0811 cranfield0219 cranfield0518 cranfield1088 cranfield0560 
[2.5680012843948976, 2.5680012843948976, 2.5680012843948976, 1.8352839030468309, 1.696259250546011]

----------------------

Results for scheme : raw_count
cranfield0094 cranfield0811 cranfield1088 cranfield0562 cranfield1082 
[6.785037002184044, 5.136002568789795, 3.6705678060936617, 3.392518501092022, 3.392518501092022]

----------------------

Results for scheme : term_frequency
cranfield0470 cranfield0389 cranfield1088 cranfield0939 cranfield0460 
[0.015145171879875099, 0.012472494489308903, 0.00812072523472049, 0.007853052085861161, 0.007439733555026364]

----------------------

Results for scheme : log_normal
cranfield0811 cranfield0094 c